# TFIDF retrieval

작성일자: 210116\
작성자: 조진욱\
목표: sklearn 과 scipy, nltk 를 가지고 passage retrieval 모델을 만들어보자\
순서: 
1. 
먼저 현재 보유한 knowledge resource 인 'test_context.json'를 tfidf 로 vectorize 해둠.
저장안되어있다면 저장. 저장이 되어있다면 불러오기
2. 
query 가 나열되어있는 json 형태 파일 'test_qas.json' 파일이 들어오면
각 query 마다 가장 cosine distance 가 가까운 context 를 찾음
3. 
그렇게 context, query pair(실제로는 정답확인을 위해 answer 까지)들을 저장해서
retrived_test.csv 로 저장함. 이는 조금 뒤에 bert 모델이 사용하게 될 예정.

비고:
1. faiss 는 원래 sparse 에 사용하면 더 좋겠지만, 처음 retrieval 설명하는 자료라서 간단히 sorted 함수로 진행
2. 파일 읽고 쓰는 함수는 모두 util 에 존재함
3. retrieval 은 topk 로 바꿀 수 있음. 기본값 topk=1

reference
retrieval code는 정민수(4seaday)코드를 참고함(비공개)

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

from scipy import spatial
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer

from tqdm.notebook import trange, tqdm
import pandas as pd
import pickle
import json
import os

In [ ]:
from utils import read_file, save_json

In [13]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [16]:
class SparseRetrieval:
    def __init__(self, mode, data_path=None):
        if not data_path:
            self.data_path = cfg.squad_dir
        else:
            self.data_path = data_path
        self.mode = mode

    def make_embedding(self, context_name):
        # Pickle save.
        pickle_name = "sparse"+ "_" + self.mode + "_embedding.bin"
        pickle_path = os.path.join(cfg.sparse_dir, pickle_name)

        context_dict = read_file(os.path.join(self.data_path, context_name))
        context = context_dict['text']

        tfidfv = TfidfVectorizer(tokenizer=self.tokenize, ngram_range=(1,2)).fit(context)

        if os.path.isfile(pickle_path):
            with open(pickle_path, "rb") as file:
                context_embeddings = pickle.load(file)
            print("Embedding pickle load.")
        else:
            context_embeddings = tfidfv.transform(context).toarray()
            # Pickle save.
            with open(pickle_path, "wb") as file:
                pickle.dump(context_embeddings, file)
            print("Embedding pickle saved.")
        return tfidfv, context_embeddings

    def sparse_searching(self, sparse_query, sparse_embedding, texts, topk=1):
        distances = spatial.distance.cdist(sparse_query, sparse_embedding, "cosine")[0]
        result = zip(range(len(distances)), distances)
        result = sorted(result, key=lambda x: x[1])

        cand_dict = {}
        candidate = []
        cand_ids = []
        for idx, distances in result[0:topk]: # top k
            candidate.append(texts[idx])
            cand_ids.append(idx)

        cand_dict['text'] = candidate
        cand_dict['ids'] = cand_ids
        return cand_dict

    def tokenize(self, text):
        stemmer = PorterStemmer()
        tokens = [word for word in word_tokenize(text)]
        stems = [stemmer.stem(item) for item in tokens]
        return stems

    def retrieve(self, model, sparse_embedding, qas_filename, topk=1):
        # 나중에 비교를 위해 context 정보도 같이 저장
        context_dict = read_file(os.path.join(self.data_path, self.mode + "_context.json"))
        context = context_dict['text']
        context_id = context_dict['ids']

        # make retrieved result as dataframe
        que, que_id, anss, ctxs, ctx_ids = [], [], [], [], []

        # load qas file.
        qas = json.load(open(os.path.join(self.data_path, qas_filename)))['data']
        for item in tqdm(qas):
            query = item['question']
            query_id = item['id']
            answers = item['answers'][0]
            query_s_embedding = model.transform([query]).toarray()
            predict_dict = self.sparse_searching(query_s_embedding,
                                                 sparse_embedding,
                                                 context,
                                                 topk)
            que.append(query)
            que_id.append(query_id)
            anss.append(answers)
            ctxs.append(predict_dict['text'])

            tmp_ctx_ids = [context_id[predict_dict['ids'][i]] for i in range(len(predict_dict['ids']))] # retrieved 된 top k 개 context 들
            ctx_ids.append(tmp_ctx_ids)

        cqas = pd.DataFrame({
            'question': que,
            'que_id': que_id,
            'answers': anss,
            'context': ctxs,          # retrieved documents
            'context_id': ctx_ids,
        })
        return cqas

In [17]:
import config as cfg
mode = 'dev'
context_file = 'dev_context.json'
qa_file =  'dev_qa.json'

ret_sp = SparseRetrieval(mode, data_path=cfg.squad_dir)
tfidfv, context_embeddings = ret_sp.make_embedding(context_file)


In [18]:
# knowledge base에 있는 articles(context) 들의 정보를 임베딩해둠
tfidfv, context_embeddings = ret_sp.make_embedding(context_file)
cqa_df = ret_sp.retrieve(tfidfv, context_embeddings, qa_file)

Embedding pickle load.


In [19]:
res_path = os.path.join(cfg.sparse_dir, f"retrieved_{mode}_sparse.csv")
cqa_df.to_csv(res_path, sep='\t', index=False)